# [모듈 2.1] 로컬 머신에서 훈련 (1)
### 데이터 인터넷에서 다운로드 및 훈련 이미지를 ECR에서 다운로드

- 필요 리소스
    - Sagemaker Session
    - AWS IAM Role
    - 로컬 머신에 Docker 설치

---
### 참조 자료
- Setting up Amazon SageMaker Environment On Your Local Machine
    - 아래 블로그에서 로컬 환경 세팅만 진행
    - https://towardsdatascience.com/setting-up-amazon-sagemaker-environment-on-your-local-machine-7329e0178adc
- Train with Amazon SageMaker on your local machine
    - 환경 셋업이 완료된 후에 코드만 수행
    - https://www.youtube.com/watch?v=K3ngZKF31mc

In [1]:
import pandas as pd
import boto3
import sagemaker

In [2]:
print(pd.__version__)
print(boto3.__version__)
print(sagemaker.__version__)

1.2.4
1.17.98
2.46.0


## 세이지 메이커 세션 및 Role 설정

In [3]:
sagemaker_session= sagemaker.Session()
role='arn:aws:iam::057716757052:role/local_machine_sagemaker_gsmoon'



In [4]:
print(sagemaker_session.account_id())
print(sagemaker_session.get_caller_identity_arn())


057716757052


Couldn't call 'get_role' to get Role ARN from role name Administrator to get Role path.


arn:aws:iam::057716757052:user/Administrator


## 데이터를 인터넷에서 다운로드

In [5]:
import os
import keras
import numpy as np
from keras.datasets import fashion_mnist

In [6]:
(x_train, y_train), (x_val, y_val) = fashion_mnist.load_data()

## 로컬에 데이터 저장

In [7]:
os.makedirs("./data", exist_ok=True)
np.savez('./data/training', image=x_train, label=y_train)
np.savez('./data/validation', image=x_val, label=y_val)


In [8]:
training_input_path = 'file://data/training.npz'
validation_input_path = 'file://data/validation.npz'
output_path = 'file:///tmp/model'


## 훈련 Estimator 정의 및 로컬 모드 실행

In [9]:
from sagemaker.tensorflow import TensorFlow

tf_estimator = TensorFlow(entry_point='mnist_keras_tf.py',
                          role=role,
                          instance_count=1, 
                          instance_type='local',   # Train on the local CPU ('local_gpu' if it has a GPU)
                          framework_version='1.15', 
                          py_version='py3',
                          hyperparameters={'epochs': 1},
                          output_path=output_path
                         )

In [10]:
# Train! This will pull (once) the SageMaker CPU/GPU container for TensorFlow to your local machine.
# Make sure that Docker is running and that docker-compose is installed

tf_estimator.fit({'training': training_input_path, 'validation': validation_input_path})

Creating 9o0o340yw5-algo-1-ex34y ... 
Creating 9o0o340yw5-algo-1-ex34y ... done
Docker Compose is now in the Docker CLI, try `docker compose up`

Attaching to 9o0o340yw5-algo-1-ex34y
9o0o340yw5-algo-1-ex34y | WARNING:tensorflow:From /usr/local/lib/python3.6/site-packages/tensorflow_core/__init__.py:1473: The name tf.estimator.inputs is deprecated. Please use tf.compat.v1.estimator.inputs instead.
9o0o340yw5-algo-1-ex34y | 
9o0o340yw5-algo-1-ex34y | 2021-06-22 08:18:19,239 sagemaker-training-toolkit INFO     Imported framework sagemaker_tensorflow_container.training
9o0o340yw5-algo-1-ex34y | 2021-06-22 08:18:19,249 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
9o0o340yw5-algo-1-ex34y | 2021-06-22 08:18:19,284 botocore.credentials INFO     Found credentials in environment variables.
9o0o340yw5-algo-1-ex34y | 2021-06-22 08:18:19,486 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
9o0o340yw5-algo-1-ex34y | 2021-06-22 0

In [11]:
!docker images | grep tensorflow

763104351884.dkr.ecr.ap-northeast-2.amazonaws.com/tensorflow-training   1.15-cpu-py3   9b598117e095   2 months ago    1.94GB


In [12]:
!tar tvfz /tmp/model/model.tar.gz

drwxr-xr-x  0 moongons 1896053708  0 Jun 22 17:19 model/
drwxr-xr-x  0 moongons 1896053708  0 Jun 22 17:19 model/1/
-rw-r--r--  0 moongons 1896053708 218515 Jun 22 17:19 model/1/saved_model.pb
drwxr-xr-x  0 moongons 1896053708      0 Jun 22 17:19 model/1/variables/
-rw-r--r--  0 moongons 1896053708 19520132 Jun 22 17:19 model/1/variables/variables.data-00000-of-00001
-rw-r--r--  0 moongons 1896053708     1500 Jun 22 17:19 model/1/variables/variables.index
